In [1]:
import requests

In [2]:
URL = "https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"
page=requests.get(URL)

In [11]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page.content, 'html.parser')

In [55]:
# Get main content <div>
poke_content=soup.find(id='mw-content-text')

# Get all <table> elements
poke_tables=poke_content.find_all('table')[1:8]
poke_gens = poke_content.find_all(class_='mw-headline')[1:-1]

link = "https://bulbapedia.bulbagarden.net/wiki"

In [104]:
generations = []

for index in range(len(poke_tables)):
    table = poke_tables[index]
    gen_list = []
    
    for i in range(3, len(table.contents), 2):    
        poke_info = table.contents[i]
        
        kdex=poke_info.contents[1].text.strip()
        ndex=poke_info.contents[3].text.strip()
        name=poke_info.contents[7].text.strip()
        type1=poke_info.contents[9].text.strip()
        url= link + poke_info.find('a')['href']
        
        if kdex == '':
            kdex = gen_list[-1]['kdex']
        
        if len(poke_info.contents) > 10:
            type2=poke_info.contents[11].text.strip()
            
            gen_list.append({
                "kdex": kdex,
                "ndex": ndex,
                "name": name,
                "type1": type1,
                "type2": type2,
                "url": url
            })
            
        else:             
            gen_list.append({
                "kdex": kdex,
                "ndex": ndex,
                "name": name,
                "type1": type1,
                "url": url
            })
            
    generations.append({
        "Generation": poke_gens[index].text.strip(),
        "Pokemons": gen_list
    })

In [93]:
for gen in generations:
    with open(f'Pokemon {gen["Generation"]}.json', 'w') as outfile:
        json.dump(gen['Pokemons'], outfile)

In [105]:
all_gen = []

for gen in generations:
    for poke in gen['Pokemons']:
        all_gen.append(poke)
        all_gen[-1]['generation'] = gen['Generation']

In [107]:
with open('Pokemon All Generations.json', 'w') as outfile:
    json.dump(generations, outfile)